In [1]:
import socket
import ipaddress
import netifaces

def get_broadcast_address():
    """Get the broadcast address for the primary network interface"""
    try:
        # Get default gateway interface
        default_gateway = netifaces.gateways()['default'][netifaces.AF_INET][1]
        addrs = netifaces.ifaddresses(default_gateway)
        
        # Get IPv4 address and netmask
        ip = addrs[netifaces.AF_INET][0]['addr']
        netmask = addrs[netifaces.AF_INET][0]['netmask']
        
        # Calculate network address
        network = ipaddress.IPv4Network(f'{ip}/{netmask}', strict=False)
        
        # Return broadcast address as string
        return str(network.broadcast_address)
    except Exception as e:
        print(f"Error getting broadcast address: {e}")
        # Fallback to common local network broadcast addresses
        common_broadcasts = ["192.168.1.255", "192.168.0.255", "10.0.0.255"]
        for addr in common_broadcasts:
            try:
                # Try to create a socket to test if this network exists
                sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
                sock.bind(('0.0.0.0', 0))
                sock.sendto(b'', (addr, 38899))
                sock.close()
                return addr
            except:
                continue
        return "255.255.255.255"  # Last resort fallback
get_broadcast_address()

'192.168.0.255'

In [2]:
import asyncio
import nest_asyncio
from pywizlight import wizlight, discovery, PilotBuilder

# This allows asyncio.run() to work in Jupyter
nest_asyncio.apply()


async def discover_lights(broadcast_space:str | None =None):
    """Discover WiZ lights on the network"""
    if broadcast_space is None:
        broadcast_space="255.255.255.255"
    bulbs = await discovery.discover_lights(broadcast_space=broadcast_space)
    print(f"Found {len(bulbs)} WiZ lights:")
    for bulb in bulbs:
        print(f"IP: {bulb.ip}")
    return bulbs

async def connect_to_light(ip_address):
    """Connect to a specific WiZ light"""
    light = wizlight(ip_address)
    try:
        # Get the current state of the light
        state = await light.updateState()
        print(f"Successfully connected to light at {ip_address}")
        print(f"Current state: {'ON' if state.get_state() else 'OFF'}")
        return light
    except Exception as e:
        print(f"Error connecting to light: {e}")
        return None

async def main(broadcast_space:str|None=None):
    data = {
        'bulbs' : None,
        'light' : None,
    }
    
    # First discover lights on the network
    bulbs = await discover_lights(broadcast_space=broadcast_space)
    if not bulbs:
        print("No WiZ lights found on the network")
        return data
    data['bulbs'] = bulbs
    
    # Example: Connect to the first light found
    first_light = bulbs[0]
    light = await connect_to_light(first_light.ip)
    
    data['light'] = light
    
    if light:
        # Example: Turn the light on
        await light.turn_on(PilotBuilder())
        print("Light turned on")
        
        # Wait for a moment
        await asyncio.sleep(2)
        
        # Example: Set brightness to 50%
        await light.turn_on(PilotBuilder(brightness=50))
        print("Brightness set to 50%")
    return data

In [3]:
get_broadcast_address()

'192.168.0.255'

In [4]:
# Now this will work
broadcast_space='192.168.0.255'
data = asyncio.run(main(broadcast_space=broadcast_space))

Found 4 WiZ lights:
IP: 192.168.0.146
IP: 192.168.0.130
IP: 192.168.0.134
IP: 192.168.0.128
Successfully connected to light at 192.168.0.146
Current state: ON
Light turned on
Brightness set to 50%


In [32]:
import numpy as np
import panel as pn
pn.extension()

class PanelWizColorPicker:
    def __init__(self, sync_light):
        self.light = sync_light
        
        # Create color picker
        self.color_picker = pn.widgets.ColorPicker(
            name='Pick a color',
            value='#ff0000'
        )
        
        # Create brightness slider
        self.brightness = pn.widgets.IntSlider(
            name='Brightness',
            start=1,
            end=255,
            value=100
        )
        
        # Create button
        self.apply_button = pn.widgets.Button(
            name='Apply Color',
            button_type='primary'
        )
        
        # Create status text
        self.status = pn.widgets.StaticText(value='Ready')
        
        # Button click handler
        def on_click(event):
            try:
                self.light.set_hex_color(self.color_picker.value)
                self.light.set_brightness(self.brightness.value)
                self.status.value = f"Set color to {self.color_picker.value} with brightness {self.brightness.value}"
            except Exception as e:
                self.status.value = f"Error: {e}"
        
        self.apply_button.on_click(on_click)
        
        # Create layout
        self.panel = pn.Column(
            "## WiZ Light Control",
            self.color_picker,
            self.brightness,
            self.apply_button,
            self.status
        )
    
    def display(self):
        return self.panel

In [37]:
import asyncio
from pywizlight import wizlight, PilotBuilder
from functools import wraps

class WizColors:
    # Predefined color constants
    COLORS = {
        'RED': '#FF0000',
        'GREEN': '#00FF00',
        'BLUE': '#0000FF',
        'WHITE': '#FFFFFF',
        'WARM_WHITE': '#FF9933',
        'COOL_WHITE': '#F5FFFA',
        'PURPLE': '#800080',
        'PINK': '#FFC0CB',
        'ORANGE': '#FFA500',
        'YELLOW': '#FFFF00',
        'CYAN': '#00FFFF',
        'MAGENTA': '#FF00FF',
        'LAVENDER': '#E6E6FA',
        'MINT': '#98FF98',
        'CORAL': '#FF7F50',
        'TURQUOISE': '#40E0D0',
    }

    @staticmethod
    def hex_to_rgb(hex_color: str) -> tuple[int, int, int]:
        """Convert hex color string to RGB tuple."""
        # Remove the '#' if present
        hex_color = hex_color.lstrip('#')
        
        # Handle both short (e.g., #FFF) and long (#FFFFFF) hex formats
        if len(hex_color) == 3:
            hex_color = ''.join(c + c for c in hex_color)
        
        # Convert to RGB values
        r = int(hex_color[0:2], 16)
        g = int(hex_color[2:4], 16)
        b = int(hex_color[4:6], 16)
        
        return (r, g, b)

    @staticmethod
    def rgb_to_hex(r: int, g: int, b: int) -> str:
        """Convert RGB values to hex color string."""
        return f'#{r:02x}{g:02x}{b:02x}'

    @staticmethod
    def get_color(color_name: str) -> tuple[int, int, int]:
        """Get RGB values for a predefined color name."""
        color_hex = WizColors.COLORS.get(color_name.upper())
        if not color_hex:
            raise ValueError(f"Color {color_name} not found. Available colors: {', '.join(WizColors.COLORS.keys())}")
        return WizColors.hex_to_rgb(color_hex)



class WizLightSync:
    def __init__(self, async_light):
        """Initialize with an existing wizlight object"""
        self.light = async_light
        self.colors = WizColors()
        
    def _run_async(self, coro):
        """Helper method to run async code synchronously"""
        try:
            return asyncio.get_event_loop().run_until_complete(coro)
        except RuntimeError:  # If there's no event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            return loop.run_until_complete(coro)

    def apply_command(self, answer):
        """Apply a command from the JSON answer format"""
        if answer.get('is_relative', False):
            self._apply_relative_command(answer)
        else:
            self._apply_absolute_command(answer)
    
    def _apply_absolute_command(self, answer):
        """Apply absolute values from command"""
        try:
            # Get the values from the answer
            hex_color = answer.get('hex')
            brightness = answer.get('brightness')
            
            # If we have a color, set it
            if hex_color is not None:
                self.set_hex_color(hex_color)
            
            # If we have brightness, set it
            if brightness is not None:
                self.set_brightness(brightness)
                
            print(f"Applied command: {answer.get('speech')} - {answer.get('explanation')}")
            
        except Exception as e:
            print(f"Error applying command: {e}")

    def _apply_relative_command(self, answer):
        """Apply relative changes from command"""
        try:
            # Get current state
            state = self.get_state()
            
            # Handle relative brightness change
            if 'relative_brightness_multiplier' in answer:
                current_brightness = state.get_brightness()
                if current_brightness is not None:
                    # Calculate new brightness
                    new_brightness = int(current_brightness * answer['relative_brightness_multiplier'])
                    # Ensure it stays within bounds
                    new_brightness = max(1, min(255, new_brightness))
                    # Apply new brightness
                    self.set_brightness(new_brightness)
                    print(f"Changed brightness from {current_brightness} to {new_brightness}")
                else:
                    # Fallback to absolute brightness if we can't get current state
                    fallback_brightness = answer.get('brightness', 50)
                    self.set_brightness(fallback_brightness)
                    print(f"Couldn't get current brightness, set to fallback: {fallback_brightness}")
            
            print(f"Applied relative command: {answer.get('speech')} - {answer.get('explanation')}")
            
        except Exception as e:
            print(f"Error applying relative command: {e}")
    
    def turn_on(self, brightness=None, color_temp=None, rgb=None, speed=None):
        """Turn the light on with optional parameters"""
        pilot = PilotBuilder(
            brightness=brightness,
            colortemp=color_temp,
            rgb=rgb,
            speed=speed
        )
        return self._run_async(self.light.turn_on(pilot))
    
    def turn_off(self):
        """Turn the light off"""
        return self._run_async(self.light.turn_off())
    
    def get_state(self):
        """Get the current state of the light"""
        return self._run_async(self.light.updateState())
    
    def set_brightness(self, brightness):
        """Set brightness (1-255)"""
        return self.turn_on(brightness=brightness)
    
    def set_color_temp(self, color_temp):
        """Set color temperature (2200-6500)"""
        return self.turn_on(color_temp=color_temp)
    
    def set_rgb(self, r, g, b):
        """Set RGB color"""
        return self.turn_on(rgb=(r, g, b))
    
    def set_scene(self, scene):
        """Set a scene"""
        pilot = PilotBuilder(scene=scene)
        return self._run_async(self.light.turn_on(pilot))

    def set_hex_color(self, hex_color: str):
        """Set light color using hex color string (e.g., '#FF0000' or 'FF0000' or '#F00')"""
        r, g, b = WizColors.hex_to_rgb(hex_color)
        return self.set_rgb(r, g, b)
    
    def set_named_color(self, color_name: str):
        """Set light color using predefined color name"""
        r, g, b = WizColors.get_color(color_name)
        return self.set_rgb(r, g, b)

# Example usage:
def create_sync_light(async_light):
    return WizLightSync(async_light)

# Predefined scenes for easy access
WIZ_SCENES = {
    'ocean': 1,
    'romance': 2,
    'sunset': 3,
    'party': 4,
    'fireplace': 5,
    'cozy': 6,
    'forest': 7,
    'pastel_colors': 8,
    'wake_up': 9,
    'bedtime': 10,
    'warm_white': 11,
    'daylight': 12,
    'cool_white': 13,
    'night_light': 14,
    'focus': 15,
    'relax': 16,
    'true_colors': 17,
    'tv_time': 18,
    'plantgrowth': 19,
    'spring': 20,
    'summer': 21,
    'fall': 22,
    'deepdive': 23,
    'jungle': 24,
    'mojito': 25,
    'club': 26,
    'christmas': 27,
    'halloween': 28,
    'candlelight': 29,
    'golden_white': 30,
    'pulse': 31,
    'steampunk': 32,
}

# Sync light

In [38]:
# Assuming you have your light from the previous async code in 'data'
light = data['light']

# Create a sync wrapper
sync_light = WizLightSync(light)


In [57]:
commands = {
    "version": "1.0",
    "description": "Natural language to WiZ light settings converter",
    "answers": [
        {
            "speech": "make the lights cozy for our night",
            "hex": "#FF9133",
            "brightness": 40,
            "explanation": "Setting a warm amber color at low brightness to create a cozy, intimate atmosphere perfect for evening relaxation. This color temperature (~2700K) mimics warm candlelight."
        },
        {
            "speech": "set the mood",
            "hex": "#FF3366",
            "brightness": 30,
            "explanation": "Setting a soft rose-pink hue at very low brightness to create an intimate, romantic atmosphere. This color combines warmth with a subtle passionate undertone."
        },
        {
            "speech": "its raining out make it happy in here",
            "hex": "#FFE87C",
            "brightness": 200,
            "explanation": "Setting a bright, sunny yellow at high brightness to create an uplifting, cheerful atmosphere that mimics sunlight. This warm, energetic color helps counter the gray weather outside."
        },
        {
            "speech": "dim the lights",
            "hex": None,
            "brightness": 50,
            "explanation": "Reducing brightness to 50% of current level while maintaining the existing color. This assumes the lights were at a higher brightness and responds only to the dimming request.",
            "is_relative": True,
            "relative_brightness_multiplier": 0.5
        },
        {
            "speech": "jungle colors",
            "hex": "#458B00",
            "brightness": 180,
            "explanation": "Setting a rich forest green at high brightness to create a lush jungle atmosphere. This deep green color evokes the feeling of being surrounded by tropical foliage.",

        }
    ]
}

In [58]:
answers = commands['answers']
n = 0
answer = answers[n]
print(answer)
sync_light.apply_command(answer=answer)

{'speech': 'make the lights cozy for our night', 'hex': '#FF9133', 'brightness': 40, 'explanation': 'Setting a warm amber color at low brightness to create a cozy, intimate atmosphere perfect for evening relaxation. This color temperature (~2700K) mimics warm candlelight.'}
Applied command: make the lights cozy for our night - Setting a warm amber color at low brightness to create a cozy, intimate atmosphere perfect for evening relaxation. This color temperature (~2700K) mimics warm candlelight.


In [59]:
answers = commands['answers']
n = 1
answer = answers[n]
print(answer)
sync_light.apply_command(answer=answer)

{'speech': 'set the mood', 'hex': '#FF3366', 'brightness': 30, 'explanation': 'Setting a soft rose-pink hue at very low brightness to create an intimate, romantic atmosphere. This color combines warmth with a subtle passionate undertone.'}
Applied command: set the mood - Setting a soft rose-pink hue at very low brightness to create an intimate, romantic atmosphere. This color combines warmth with a subtle passionate undertone.


In [60]:
answers = commands['answers']
n = 2
answer = answers[n]
print(answer)
sync_light.apply_command(answer=answer)

{'speech': 'its raining out make it happy in here', 'hex': '#FFE87C', 'brightness': 200, 'explanation': 'Setting a bright, sunny yellow at high brightness to create an uplifting, cheerful atmosphere that mimics sunlight. This warm, energetic color helps counter the gray weather outside.'}
Applied command: its raining out make it happy in here - Setting a bright, sunny yellow at high brightness to create an uplifting, cheerful atmosphere that mimics sunlight. This warm, energetic color helps counter the gray weather outside.


In [61]:
answers = commands['answers']
n = 4
answer = answers[n]
print(answer)
sync_light.apply_command(answer=answer)

{'speech': 'jungle colors', 'hex': '#458B00', 'brightness': 180, 'explanation': 'Setting a rich forest green at high brightness to create a lush jungle atmosphere. This deep green color evokes the feeling of being surrounded by tropical foliage.'}
Applied command: jungle colors - Setting a rich forest green at high brightness to create a lush jungle atmosphere. This deep green color evokes the feeling of being surrounded by tropical foliage.


In [63]:
import time
base_color = '#3498da'
sync_light.set_hex_color(base_color)
for value in np.linspace(1,255, 100):
    sync_light.set_brightness(value)
    time.sleep(0.1)

In [29]:
sync_light.set_color_temp(color_temp=4000)

In [ ]:
sync_light.turn_off()  # Turn off

In [ ]:
sync_light.turn_on()  # Simple on

In [23]:
sync_light.set_rgb(255, 0, 0)  # Red color

In [27]:
state = sync_light.get_state()
state.get_colortemp()

In [ ]:

# Now you can control it synchronously!
sync_light.turn_on()  # Simple on
sync_light.set_brightness(50)  # 50% brightness
sync_light.set_rgb(255, 0, 0)  # Red color
sync_light.set_scene('ocean')  # Ocean scene
sync_light.turn_off()  # Turn off

# Get the current state
state = sync_light.get_state()